In [0]:
import torch
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torch.optim.optimizer import Optimizer, required
import copy
from sklearn.linear_model import LinearRegression
from torch.autograd import Variable
import os
import random

In [0]:
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

In [0]:
class SGD(Optimizer):
    r"""Implements stochastic gradient descent (optionally with momentum).

    Nesterov momentum is based on the formula from
    `On the importance of initialization and momentum in deep learning`__.

    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate
        momentum (float, optional): momentum factor (default: 0)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        dampening (float, optional): dampening for momentum (default: 0)
        nesterov (bool, optional): enables Nesterov momentum (default: False)

    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()

    __ http://www.cs.toronto.edu/%7Ehinton/absps/momentum.pdf

    .. note::
        The implementation of SGD with Momentum/Nesterov subtly differs from
        Sutskever et. al. and implementations in some other frameworks.

        Considering the specific case of Momentum, the update can be written as

        .. math::
                  v = \rho * v + g \\
                  p = p - lr * v

        where p, g, v and :math:`\rho` denote the parameters, gradient,
        velocity, and momentum respectively.

        This is in contrast to Sutskever et. al. and
        other frameworks which employ an update of the form

        .. math::
             v = \rho * v + lr * g \\
             p = p - v

        The Nesterov version is analogously modified.
    """
    #Add heavy ball flag
    def __init__(self, params, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False, heavy_ball=False):
        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        
        #add heavy ball into param list
        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov, 
                        heavy_ball=heavy_ball)
        
        #check heavy ball flag
        if heavy_ball and (momentum <= 0 or dampening != 0):
            raise ValueError("Heavy-ball momentum requires a momentum and zero dampening")

        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(SGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(SGD, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)
            group.setdefault('heavy_ball', False)

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']
            #新增读取heavy ball
            heavy_ball = group['heavy_ball']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                        #init prev momentum space in gpu
                        param_state['prev_momentum_buffer'] = torch.zeros(buf.size()).cuda()
                    else:
                        buf = param_state['momentum_buffer']
                        #heavy ball iteration
                        prev_buf = param_state['prev_momentum_buffer']
                        if heavy_ball:
                            temp = buf
                            buf.add_(-prev_buf).mul_(momentum).add_(1 - dampening, d_p)
                            param_state['prev_momentum_buffer'] = temp
                        else:
                            buf.mul_(momentum).add_(1 - dampening, d_p)
                    if nesterov:
                        d_p = d_p.add(momentum, buf)
                    else:
                        d_p = buf

                p.data.add_(-group['lr'], d_p)

        return loss

In [0]:
class AccSGD(Optimizer):
    r"""Implements the algorithm proposed in https://arxiv.org/pdf/1704.08227.pdf, which is a provably accelerated method 
    for stochastic optimization. This has been employed in https://openreview.net/forum?id=rJTutzbA- for training several 
    deep learning models of practical interest. This code has been implemented by building on the construction of the SGD 
    optimization module found in pytorch codebase.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        lr (float): learning rate (required)
        kappa (float, optional): ratio of long to short step (default: 1000)
        xi (float, optional): statistical advantage parameter (default: 10)
        smallConst (float, optional): any value <=1 (default: 0.7)
    Example:
        >>> from AccSGD import *
        >>> optimizer = AccSGD(model.parameters(), lr=0.1, kappa = 1000.0, xi = 10.0)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    """

    def __init__(self, params, lr=0.001, kappa = 1000.0, xi = 10.0, smallConst = 0.7, weight_decay=0):
        defaults = dict(lr=lr, kappa=kappa, xi=xi, smallConst=smallConst,
                        weight_decay=weight_decay)
        super(AccSGD, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AccSGD, self).__setstate__(state)

    def step(self, closure=None):
        """ Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            large_lr = (group['lr']*group['kappa'])/(group['smallConst'])
            Alpha = 1.0 - ((group['smallConst']*group['smallConst']*group['xi'])/group['kappa'])
            Beta = 1.0 - Alpha
            zeta = group['smallConst']/(group['smallConst']+Beta)
            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                param_state = self.state[p]
                if 'momentum_buffer' not in param_state:
                    param_state['momentum_buffer'] = copy.deepcopy(p.data)
                buf = param_state['momentum_buffer']
                buf.mul_((1.0/Beta)-1.0)
                buf.add_(-large_lr,d_p)
                buf.add_(p.data)
                buf.mul_(Beta)

                p.data.add_(-group['lr'],d_p)
                p.data.mul_(zeta)
                p.data.add_(1.0-zeta,buf)

        return loss

In [0]:


os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='0'

#generate gaussian distribution data
def get_batch(N,k,w):    
    X = torch.distributions.multivariate_normal.MultivariateNormal(
        loc=torch.Tensor([0,0]), 
        covariance_matrix=torch.Tensor([[1., 0.],[0., 1/k]])).sample((N,))
    y = X@w+0.1 * torch.randn(N, 1)
    if torch.cuda.is_available(): 
        return Variable(X).cuda(),Variable(y).cuda()
    else:
        return Variable(X),Variable(y)

#generate discrete distribution data      
def discrete(N,k,w):
  X=torch.zeros(N,2)
  for i in range(0,N):
    weights = torch.tensor([0.5,0.5])
    dist=torch.multinomial(input=weights,num_samples=2).float()
    X[i]=dist
  X[:,1]=X[:,1].float()*(2.0/k)
  y= X@w  
  if torch.cuda.is_available(): 
        return Variable(X).cuda(),Variable(y).cuda()
  else:
        return Variable(X),Variable(y)

# linear model
class LinearRegression_(nn.Module):
    def __init__(self):
        super(LinearRegression_,self).__init__()
        self.linear = nn.Linear(2,1)
    def forward(self,input):
        output = self.linear(input)
        return output    

In [0]:
# learning rate and momentum recording object
class Rate(object):
  def __init__(self, lr, mm, err):
        self.lr = lr
        self.mm = mm
        self.err = err
        
  def getRate(self):
    return self.lr, self.mm
  
  def getErr(self):
    return self.err

In [0]:
# train one epoch
def train(iteration, model, optimizer, criterion, X, y, lr, mm, k, momentum=False):
  sub_loss_list=[]
  init_loss = 0
  for t in range(0, iteration):
    output = model(X)
    loss = criterion(output,y)
    loss_value = loss.data
    
    if t == 1:
      init_loss = loss_value
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    elif t > iteration/2:
      if loss.data < init_loss:
        r = Rate(lr, mm, loss.data.cpu().numpy())
        sub_loss_list.append(r)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      else:
        print(loss.data, init_loss)
        if momentum:
          print("Not converge at learning rate: ",lr," momentum: ",mm," k: ",k, " epoch: ",t, "iter: ",iteration)
        else:
          print("Not converge at learning rate: ",lr," k: ",k, " epoch: ",t, "iter: ",iteration)
        return []
    else:
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
#   if momentum:
#     print("Converge at learning rate: ",lr," momentum: ",mm," k: ",k)
#   else:
#     print("Converge at learning rate: ",lr," k: ",k)
  return sub_loss_list


# check whether a combination is converged
def converge(kk = 9, lr=0.1, mm=0.9, gaussian=False, momentum=False, asgd=False, **kwargs):
  
  N=1000
  dis_w=torch.tensor([[1.5],[2]])
  
  k=torch.zeros(kk)

  final_loss=torch.zeros(kk)
  min_loss=torch.zeros(kk)
  min_loss_index=torch.zeros(kk)
  loss_list=[]

  for i in range (0,kk):
    
    k[i]= 2**(i+4)
    iteration=torch.tensor(5*k[i]).int()
    sub_loss_list=[]
    
    if gaussian:
      X, y = get_batch(N,k[i],dis_w)
    else:
      X, y = discrete(N,k[i],dis_w)

    model = LinearRegression_().cuda()
    if asgd:
        optimiser = AccSGD(model.parameters(), lr=lr)
    else:
        if momentum:
            optimiser = SGD(model.parameters(), lr=lr, momentum=mm, **kwargs)
        else:
            optimiser = SGD(model.parameters(), lr=lr, **kwargs)
            
    criterion = nn.MSELoss()
    sub_loss_list = train(iteration, model, optimiser, criterion, X, y, lr, mm, k[i], momentum=momentum)
    print("----------Sub Sub Converge List with Size: ",len(sub_loss_list))  
    if len(sub_loss_list) > 100:
      sub_loss_list = random.sample(sub_loss_list, k=100)
#     print("----------",len(sub_loss_list))
      
    for rate in sub_loss_list:
      loss_list.append(rate)
      
#     if momentum:
#       print("Train finished at learning rate: ",lr," momentum: ",mm)
#     else:
#       print("Train finished at learning rate: ",lr)
  print("----------Sub Converge List with Size: ",len(loss_list))  
  if len(loss_list) > 100:
    loss_list = random.sample(loss_list, k=100)
    
  return loss_list
  

In [0]:
# try all combinations

def grid_search(learn_rate=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0], momentum=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0], gaussian=False, mmf=False, **kwargs):
  converge_list = []
  
  for lp in range(10):
    lr = learn_rate[lp]
    if mmf:
      for mp in range(10):
        mm = momentum[mp]
        sub_converge_list = converge(lr=lr, mm=mm, momentum=mmf, gaussian=gaussian, **kwargs)
        for rate in sub_converge_list:
          converge_list.append(rate)
        print("Train finished at: (",lr,",",mm,")")
        print(len(converge_list))
    else:
      sub_converge_list = converge(lr=lr)
      print(len(sub_converge_list))
      for rate in sub_converge_list:
        converge_list.append(rate)
      print("Train finished at: (",lr,",-)")
      print(len(converge_list))
#     print((converge_list[0]).getErr())  
  print()
  
  return converge_list

In [0]:
# grid search for NAG HB ASGD SGD Momentum in guassian or discrete distribution 
nag2_dis_rates = grid_search(mmf=True, heavy_ball=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


----------Sub Sub Converge List with Size:  39
----------Sub Sub Converge List with Size:  79
----------Sub Sub Converge List with Size:  159
----------Sub Sub Converge List with Size:  319
----------Sub Sub Converge List with Size:  639
----------Sub Sub Converge List with Size:  1279
----------Sub Sub Converge List with Size:  2559
----------Sub Sub Converge List with Size:  5119
----------Sub Sub Converge List with Size:  10239
----------Sub Converge List with Size:  818
Train finished at: ( 0.1 , 0.1 )
100
----------Sub Sub Converge List with Size:  39
----------Sub Sub Converge List with Size:  79
----------Sub Sub Converge List with Size:  159
----------Sub Sub Converge List with Size:  319
----------Sub Sub Converge List with Size:  639
----------Sub Sub Converge List with Size:  1279
----------Sub Sub Converge List with Size:  2559
----------Sub Sub Converge List with Size:  5119
----------Sub Sub Converge List with Size:  10239
----------Sub Converge List with Size:  818
Train

In [0]:
# print the opt rates

minlr = 0.1
minmm = 0.9
minerr = 0

for rate in nag2_dis_rates:
  if (minerr == 0 or rate.getErr() < minerr) and rate.getErr() != 0:
#     print(rate)
    minerr = rate.getErr()
    minlr, minmm = rate.getRate()
#   print(rate.getRate(),long(rate.getErr()))

print(minlr, minmm, minerr)

0.6 0.1 1.7584404e-21
